In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

In [2]:
df = gpd.read_file("./data/LSMD_CONT_LDREG_2097_11500.shp",encoding='cp949')

In [3]:
# 좌표계 변환 정보 추가
df['lat']=0
df['lon']=0
df['polygon']=0

In [4]:
df = df[df['jibun'].str.contains('공')]
df

,pnu,jibun,bchk,sgg_oid,col_adm_se,geometry,lat,lon,polygon
2,1150010600107570002,757-2 공,1,143940,11500,"POLYGON ((184318.990 449685.770, 184348.990 44...",0,0,0
14,1150010600107570004,757-4 공,1,143959,11500,"POLYGON ((184253.520 449642.970, 184259.310 44...",0,0,0
17,1150010600107570006,757-6 공,1,143963,11500,"POLYGON ((184127.190 449604.960, 184117.240 44...",0,0,0
19,1150010600107610000,761 공,1,143965,11500,"POLYGON ((184170.860 449463.990, 184191.570 44...",0,0,0
54,1150010500107490004,749-4 공,9,390978,11500,"POLYGON ((184055.715 450459.259, 184055.713 45...",0,0,0
...,...,...,...,...,...,...,...,...,...
42798,1150010300110930053,1093-53 공,1,243233,11500,"POLYGON ((186402.147 450128.284, 186396.010 45...",0,0,0
42864,1150010400114740001,1474-1공,1,164636,11500,"POLYGON ((187081.880 451492.390, 187069.530 45...",0,0,0
42865,1150010400114710000,1471공,1,164624,11500,"POLYGON ((186843.765 451846.574, 186845.070 45...",0,0,0
42868,1150010400114780001,1478-1공,1,164645,11500,"POLYGON ((186851.640 451390.680, 186820.890 45...",0,0,0


In [5]:
import pyproj
import numpy as np
from shapely.geometry import Polygon

def geo_transform(in_lat, in_lon):
    # 좌표계 변환을 수행합니다.
    # in_proj = pyproj.Proj(init='epsg:5181')   # 기존 좌표계 (Korea 2000)
    in_proj = pyproj.Proj(init='epsg:2097')   # 기존 좌표계 (Korea 2000)
    # in_proj = pyproj.Proj(init='epsg:5186')   # 기존 좌표계 (Korea 2000)
    out_proj = pyproj.Proj(init='epsg:4326')  # 변환할 좌표계 (WGS84)
    lon,lat = pyproj.transform(in_proj, out_proj, in_lat, in_lon)
    lon+=0.0028
    return (lat,lon)

def geo_transform_2(coord):
    # 좌표계 변환을 수행합니다.
    in_proj = pyproj.Proj(init='epsg:5181')   # 기존 좌표계 (Korea 2000)
    out_proj = pyproj.Proj('EPSG:4326')  # 변환할 좌표계 (WGS84)

    coord=np.array(coord)
    
    fx, fy = pyproj.transform(in_proj, out_proj, coord[:,0], coord[:,1])
    
    arr = np.dstack([fx, fy])[0]
    arr[:,0] += 0.0027
    arr[:,1] += 0.0008
    polygon = Polygon(arr)
    lat, lon = polygon.centroid.x, polygon.centroid.y

    return lat, lon, arr
    
def transform_lat_lon(row):
    geom = row['geometry']
    # 다각형 객체를 생성합니다.
    coords = list(geom.exterior.coords)

    # 좌표변환
    lat, lon, trans = geo_transform_2(coords)

    return lat, lon, trans


In [6]:
# geometry의 lat,lon, polygon 추출해서 df에 저장
df[['lat', 'lon', 'polygon']] = df.apply(transform_lat_lon, axis=1, result_type='expand')

/Users/nowkim/opt/anaconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/4x/t45944gn70vch0nx3k1jlq700000gn/T/ipykernel_78673/1822456495.py:22: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  fx, fy = pyproj.transform(in_proj, out_proj, coord[:,0], coord[:,1])
/Users/nowkim/opt/anaconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/got

In [7]:
# 지도에 표현
# 지도 객체를 생성합니다.
# lat = 37.544776089456754
# lon = 126.81996250295592
lat = df['lat'].iloc[0]
lon = df['lon'].iloc[0]

map = folium.Map(location=[lat,lon], zoom_start=16, tiles='cartodbpositron')

for c in df['polygon']:
    c = list(c)
    folium.Polygon(locations=c, color='red', fill=True, fill_color='red', fill_opacity=0.5).add_to(map)

# 생성한 지도를 출력합니다.
map

In [8]:
# dataframe csv로 저장
df.to_csv('./data/park_road.csv', index=False)